In [ ]:
import requests
import logging
url = "https://api.bcb.gov.br/dados/serie/bcdata.sgs.10813/dados?formato=json;dataInicial=01/01/2025;dataFinal=31/12/2025&dataInicial=10/12/2025&dataFinal=10/12/2025"
    
try:
    response = requests.get(url)
    response.raise_for_status()
    
except requests.exceptions.RequestException as e:
    logging.error(f"Erro na conexão da API: {e}")